In [128]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re
from html import unescape
from datetime import datetime as dt
import time
import lxml
import itertools

# Event-Level Statistics:

In [129]:
#New Fight Details:
url = 'http://www.ufcstats.com/event-details/dfb965c9824425db'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')

In [130]:
#Set up datatype dictionary: 

data_types = {
    'R_STR': int, 
    'B_STR': int,
    'R_TD': int, 
    'B_TD': int, 
    'R_SUB': int, 
    'R_SUB': int, 
    'R_PASS': int, 
    'B_PASS': int,
    'ROUND': int,
}


def remove_space_lines(text):
    pattern1 = re.compile(r'[\s\s+]')
    return re.sub(pattern1, ' ', text)

#Determine if observation is a title-bout
def find_belt(img_tag):
    try:
        image_link = img_tag['src']
        if re.match(r'.*belt.*', image_link) != None:
            return True
    except:
        return False
    
    
def get_fight_auxiliary(soup):
    '''
    Input: beautifulsoup of an event url: (ie. http://www.ufcstats.com/event-details/53278852bcd91e11)
    Outputs: pandas Series
        date, location, attendance
    '''
    
    table = []
    
    auxiliary_table = soup.find_all('li', {'class': 'b-list__box-list-item'})
    for item in auxiliary_table:
        attribute = remove_space_lines(item.text).strip()

        #If attribute is missing, replace with ''
        try:
            attribute = re.findall(r'\s\s+(.*)', attribute)[0]
        except:
            attribute = '' 
        
        table.append(attribute)
        
    table_series = pd.Series(table)
    table_series.index = ['date', 'location', 'attendance']
    
    if table_series['attendance'] != '':
        table_series['attendance'] = re.sub(',', '', table_series['attendance'])
        table_series['attendance'] = int(table_series['attendance'])
    
    table_series['date'] = dt.strptime(table_series['date'], '%B %d, %Y').strftime('%d-%m-%Y')

    return table_series


def get_page_stats(url):
    
    #Given url of list of events, returns list of event details:
    #ie. return summary statistics for all fights on page like: http://www.ufcstats.com/event-details/53278852bcd91e11
    
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    stat_table = soup.findAll('table')[0].contents #Contents of the main table in html
    
    table_data = stat_table[3] #first 2 indices are empty strings, table_data is html starting from first table row
    detail_data = table_data.find_all('p') #within table rows, there are <p> labels for table text
    auxiliary_data = get_fight_auxiliary(soup) #Returns date, location, attendance of event
    
    image_data = table_data.find_all('img') #get image links to find belt for 
    
    contents = [] #table contents
    title_match_index = [] #used later to track which fights are title_bouts
    
    #Loop through elements of detail_data (html table) to scrape fight details:
    for index, item in enumerate(detail_data):
        
        #find image of belt == title_bout
        image = item.find('img')
        if find_belt(image):
            title_match_index.append(index) #grab index of fight in which belt appears
            
        #contents is list of all text from each element of table     
        contents.append(item.text)  

    
    #Clean up elements
    contents = list(map(lambda x: remove_space_lines(x), contents))
    contents = list(map(lambda x: x.strip(), contents)) 
    
    draw_index = []
    
    #When there's a draw or NC, additional tags are created --> remove the tag to reformat correctly   
    for i in np.arange(0, len(contents)-10, 16):

        if contents[i] != 'win':
            
            #Get the index of the match that was drawn & remove that element
            draw_index.append(np.floor_divide(i, 16)) 
            contents.pop(i)
                    
    #Extract links to more detailed fight statistics
    fight_links = table_data.find_all('a', {'class': 'b-flag b-flag_style_green'})
    fight_links = [item['href'] for item in fight_links]
    
    draw_links = table_data.find_all('a', {'class': 'b-flag b-flag_style_bordered'})
    draw_links = [item['href'] for item in draw_links]
    draw_links = list(dict.fromkeys(draw_links)) #Remove duplicate links from the drawn fights
    
    for index, link in zip(draw_index, draw_links):
        fight_links.insert(index, link)
    
    #each row of data is 16 elements: reformats 1 observation per row
    formatted_contents = np.array(contents).reshape((-1, 16))
    formatted_contents = pd.DataFrame(formatted_contents)
    
    #the first row is a list of 'wins'
    #formatted_contents.drop(0, axis = 1, inplace = True)
    
    #Run a floor_divide to put the image of the belt in the correct fight

    title_match = np.floor_divide(title_match_index, 16) 

    #Initialize title_bout column with all 0's
    titles = np.zeros(16)
    if len(title_match) != 0:
        titles[title_match] = 1
    
    title_series = pd.Series(titles)
    
    formatted_contents['title_bout'] = title_series
    
    #rename columns
    formatted_contents.columns = ['Winner', 'R_fighter', 'B_fighter', 'R_STR', 'B_STR', 
                               'R_TD', 'B_TD', 'R_SUB', 'R_SUB', 'R_PASS', 'B_PASS',
                              'WEIGHT_CLASS', 'METHOD', 'DETAIL', 'ROUND', 'TIME', 'title_bout']
    
    #convert columns to appropriate data types
    formatted_contents.replace('--', 99999, inplace = True)
    formatted_contents = formatted_contents.astype(data_types)
    formatted_contents['TIME'] = formatted_contents['TIME'].apply(lambda x: dt.strptime(x, '%M:%S').time())
    formatted_contents['link'] = fight_links
    
    return (formatted_contents, auxiliary_data)

In [131]:
event_level = get_page_stats(url)

event_level[0]['date'] = event_level[1].loc['date']
event_level[0]['location'] = event_level[1].loc['location']
event_level[0]['attendance'] = event_level[1].loc['attendance']

# Event-Detail Statistics:

In [132]:
def empty_page(soup):
    
    sections = soup.find('section', {'class': 'b-fight-details__section js-fight-section'})
    
    if sections.contents[0] == '\n    Round-by-round stats not currently available.\n': 
        return True
    return False


def empty_df():
    temp = pd.Series([99999]*len(template), index=template)
    return temp




def get_totals_table(soup):
    '''
    Input: beautifulsoup of detailed fight stats site (ie. http://www.ufcstats.com/fight-details/e0b323dae5bf4c90)
    Output: pandas DataFrame matching table labeled 'TOTALS'
    
    '''

    html_table = soup.find_all('table')[0]

    #Scrape Statistics: separated by double-space
    stats_table = pd.read_html(str(html_table))[0]
    table_columns = stats_table.columns
    total_statistics = [item.split('  ') for item in stats_table.loc[0][1:]]
    
    #Scrape Fighter names
    names_table = soup.find('td', {'class': 'b-fight-details__table-col l-page_align_left'})    
    names = [remove_space_lines(item.text).strip() for item in names_table.find_all('p')]
    
    #Append together, rearrange columns, and rename columns:
    total_statistics.append(names)

    total_statistics = pd.DataFrame(total_statistics).T
    total_statistics = total_statistics[[9,0,1,2,3,4,5,6,7,8]]
    total_statistics.columns = table_columns
    
    return total_statistics

def get_ss_table(soup):
    '''
    Input: beautifulsoup of detailed fight stats site (ie. http://www.ufcstats.com/fight-details/e0b323dae5bf4c90)
    Output: pandas DataFrame matching table labeled 'SIGNIFICANT STRIKES'
    
    '''
    html_table = soup.find_all('table')[2]

    
    #Scrape Statistics: separated by double-space
    stats_table = pd.read_html(str(html_table))[0]
    table_columns = stats_table.columns
    total_statistics = [item.split('  ') for item in stats_table.loc[0][1:]]
    
    #Append together, rearrange columns, and rename columns:
    #total_statistics.append(names)

    total_statistics = pd.DataFrame(total_statistics).T
    total_statistics.columns = table_columns[1:]
    
    return total_statistics
    
def get_combined(soup):
    name_stats = get_totals_table(soup)
    significant_stats = get_ss_table(soup)

    combined = pd.concat([name_stats, significant_stats], axis = 1)
    combined = combined.loc[:, ~combined.columns.duplicated()]
    combined.drop('Sig. str', axis = 1, inplace = True)

    return combined



def fight_attributes(soup):
        
    header_table = soup.find_all('i', {'class': 'b-fight-details__text-item'})[0:4]
    
    table = []
    for item in header_table:

        detail = remove_space_lines(item.text).strip()
        
        try:
            table.append(re.findall(r'\s\s+(.*)', detail)[0])
        except:
            table.append('')
     
    table_series = pd.Series(table, index=['rounds', 'time', 'format', 'referee'])
    #table_series = table_series.astype(data_types)
    table_series['rounds'] = int(table_series['rounds'])
    table_series['time'] = dt.strptime(table_series['time'], '%M:%S').time()
        

    return table_series




In [133]:
countables = ['Sig. str.', 'Total str.', 'Td', 'Head', 'Body', 'Leg', 'Distance', 'Clinch', 'Ground']
hit_att = ['Hits', 'Attempts']

countable_cols = list(itertools.product(countables, hit_att))
countable_cols = ['{} {}'.format(item[0], item[1]) for item in countable_cols]
#countable_cols


def split_countable(combined_df):
    
    split = combined_df.apply(lambda x: x.apply(lambda y: y.split('of')))
    
    split_stats = []
    for index, series in split.iterrows():
        split_stats.append(list(series.apply(pd.Series).stack()))
        
    split_df = pd.DataFrame(split_stats)
    split_df.columns = countable_cols
    
    return split_df
    
def df_single_row_format(combined_df, attr):
    
    df = pd.concat([combined_df.loc[0], combined_df.loc[1]], axis = 0)
    df['rounds'] = attr['rounds']
    df['time'] = attr['time']
    df['format'] = attr['format']
    df['referee'] = attr['referee']
    
    return df
    
#PUT IT ALL TOGETHER:
def get_detailed_page_stats(url):
    
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    #get Attributes:
    attributes = fight_attributes(soup)
    
    
    #If relevant statistics are unavailable return an 'unavailable_df'    
    if empty_page(soup) == True:
        empty = empty_df()   

        combined_df = pd.concat([empty, attributes], axis = 0)
        
        combined_df['url'] = url
        return combined_df
        
    
    else:
     
    #Get fight_df:

        combined_df = get_combined(soup)
        split_df = combined_df[countables]


        combined_df.drop(countables, axis = 1, inplace = True)

        countable_df = split_countable(split_df)

        combined_df = pd.concat([combined_df, countable_df], axis = 1)

        single_df = df_single_row_format(combined_df, attributes)

        single_df['url'] = url

        return single_df   
    

In [134]:
event_links = event_level[0]['link']
event_links

0     http://www.ufcstats.com/fight-details/ce074851...
1     http://www.ufcstats.com/fight-details/27bfb920...
2     http://www.ufcstats.com/fight-details/58540b24...
3     http://www.ufcstats.com/fight-details/821a1c6f...
4     http://www.ufcstats.com/fight-details/81f2f2d4...
5     http://www.ufcstats.com/fight-details/adac148e...
6     http://www.ufcstats.com/fight-details/51461da4...
7     http://www.ufcstats.com/fight-details/b1e66cea...
8     http://www.ufcstats.com/fight-details/f039c935...
9     http://www.ufcstats.com/fight-details/906283a2...
10    http://www.ufcstats.com/fight-details/ebe122e9...
Name: link, dtype: object

In [135]:
import time
fight_details = []

for index, item in enumerate(event_links):
    
    try:
        print(index)
        fight_details.append(get_detailed_page_stats(item))
        
    except:
        print("error: {}".format(item))
        fight_details.append(index)
        
    time.sleep(1)

0
1
2
3
4
5
6
7
8
9
10


In [136]:
batch_to_file = pd.DataFrame()

for item in fight_details:
    temp = item.to_frame().T
    batch_to_file = pd.concat([batch_to_file, temp], axis = 0)

In [137]:
batch_to_file

,Fighter,KD,Sig. str. %,Td %,Sub. att,Pass,Rev.,Sig. str. Hits,Sig. str. Attempts,Total str. Hits,...,Distance Attempts,Clinch Hits,Clinch Attempts,Ground Hits,Ground Attempts,rounds,time,format,referee,url
0,Alistair Overeem,1,80%,100%,0,3,0,68,85,97,...,19,1,3,18,26,2,00:03:00,5 Rnd (5-5-5-5-5),Dan Miragliotta,http://www.ufcstats.com/fight-details/ce074851...
0,Claudia Gadelha,0,40%,25%,0,1,0,84,210,92,...,170,4,4,0,2,3,00:05:00,3 Rnd (5-5-5),Keith Peterson,http://www.ufcstats.com/fight-details/27bfb920...
0,Dan Ige,0,48%,25%,0,0,0,79,164,100,...,107,8,9,18,28,3,00:05:00,3 Rnd (5-5-5),Jason Herzog,http://www.ufcstats.com/fight-details/58540b24...
0,Eryk Anders,0,42%,0%,0,0,0,41,97,73,...,99,13,17,0,0,3,00:05:00,3 Rnd (5-5-5),Herb Dean,http://www.ufcstats.com/fight-details/821a1c6f...
0,Song Yadong,0,50%,0%,0,0,0,101,201,119,...,127,20,29,4,8,3,00:05:00,3 Rnd (5-5-5),Dan Miragliotta,http://www.ufcstats.com/fight-details/81f2f2d4...
0,Matt Brown,0,54%,0%,0,0,0,33,61,33,...,62,1,2,7,9,2,00:00:18,3 Rnd (5-5-5),Jason Herzog,http://www.ufcstats.com/fight-details/adac148e...
0,Anthony Hernandez,0,0%,0%,0,0,0,0,3,0,...,7,5,5,5,6,1,00:00:39,3 Rnd (5-5-5),Michael Cardoso,http://www.ufcstats.com/fight-details/51461da4...
0,Giga Chikadze,1,45%,100%,0,0,0,61,135,61,...,94,0,0,0,0,3,00:05:00,3 Rnd (5-5-5),Keith Peterson,http://www.ufcstats.com/fight-details/b1e66cea...
0,Darren Elkins,0,47%,7%,0,1,0,121,253,142,...,206,22,28,0,0,3,00:05:00,3 Rnd (5-5-5),Herb Dean,http://www.ufcstats.com/fight-details/f039c935...
0,Cortney Casey,0,50%,0%,2,0,0,6,12,18,...,4,2,2,1,1,1,00:03:36,3 Rnd (5-5-5),Michael Cardoso,http://www.ufcstats.com/fight-details/906283a2...


# Data Clean & Combine:

In [138]:
df = batch_to_file.copy()
df.reset_index(drop = True, inplace = True)
df.columns = df.columns.where(~df.columns.duplicated(), df.columns + '.1')

In [139]:
df['Sig. str. %'] = df['Sig. str. %'].apply(lambda x: re.sub('%', '', x))
df['Td %'] = df['Td %'].apply(lambda x: re.sub('%', '', x))

df['Sig. str. %.1'] = df['Sig. str. %.1'].apply(lambda x: re.sub('%', '', x))
df['Td %.1'] = df['Td %.1'].apply(lambda x: re.sub('%', '', x))

df['Sig. str. %'] = df['Sig. str. %'].astype(float)
df['Sig. str. %'] = df['Sig. str. %'] / 100

df['Td %'] = df['Td %'].astype(float)
df['Td %'] = df['Td %'] / 100

df['Sig. str. %.1'] = df['Sig. str. %.1'].astype(float)
df['Sig. str. %.1'] = df['Sig. str. %.1'] / 100

df['Td %.1'] = df['Td %.1'].astype(float)
df['Td %.1'] = df['Td %.1'] / 100


In [140]:
combined_df = pd.concat([event_level[0], df], axis = 1)

drop_columns = ['R_STR', 'B_STR', 'R_TD', 'B_TD',
       'R_SUB', 'R_SUB', 'R_PASS', 'B_PASS']
combined_df.drop(drop_columns, axis = 1, inplace = True)
combined_df.drop(['ROUND', 'TIME', 'link'], axis = 1, inplace = True)

In [141]:
winner = []
for index, row in combined_df.iterrows():
    
    if row['Winner'] != 'win':
        winner.append(99999)
    
    elif (row['R_fighter'] == row['Fighter']):
        winner.append(0)
    else:
        winner.append(1)
        
combined_df['label'] = winner

#Reformat match format:

round_times = combined_df['format'].apply(lambda x: re.search(r'\(.*\)', x))
round_times = round_times.apply(lambda x: x.group().strip('()').split('-') if x != None else [1])
round_times = round_times.apply(lambda x: np.array(x, dtype = 'float'))
round_times[0:5]

#Returns time in minutes given datetime object:
def to_minutes(time):
    
    try:
        set_time = dt.strptime(time, '%H:%M:%S')
    except:
        pass
    
    return time.minute + time.second / 60

#Calculate total match time
round_dot = []

for num_rounds, time, item in zip(combined_df['rounds'], combined_df['time'], round_times):
    
    minutes = to_minutes(time)
    
    round_indicator = np.zeros(len(item))
    round_indicator[0: num_rounds-1] = 1
    
    round_time = round(np.dot(round_indicator, item) + minutes, 2)
    
    round_dot.append(round_time)
    
combined_df['match_time'] = round_dot

combined_df.reset_index(drop = True, inplace = True)
combined_df

,Winner,R_fighter,B_fighter,WEIGHT_CLASS,METHOD,DETAIL,title_bout,date,location,attendance,...,Clinch Attempts.1,Ground Hits.1,Ground Attempts.1,rounds,time,format,referee,url,label,match_time
0,win,Alistair Overeem,Walt Harris,Heavyweight,KO/TKO,Punches,0.0,16-05-2020,"Jacksonville, Florida, USA",,...,3,18,26,2,00:03:00,5 Rnd (5-5-5-5-5),Dan Miragliotta,http://www.ufcstats.com/fight-details/ce074851...,0,8.00
1,win,Claudia Gadelha,Angela Hill,Women's Strawweight,S-DEC,,0.0,16-05-2020,"Jacksonville, Florida, USA",,...,4,0,2,3,00:05:00,3 Rnd (5-5-5),Keith Peterson,http://www.ufcstats.com/fight-details/27bfb920...,0,15.00
2,win,Dan Ige,Edson Barboza,Featherweight,S-DEC,,0.0,16-05-2020,"Jacksonville, Florida, USA",,...,9,18,28,3,00:05:00,3 Rnd (5-5-5),Jason Herzog,http://www.ufcstats.com/fight-details/58540b24...,0,15.00
3,win,Krzysztof Jotko,Eryk Anders,Middleweight,U-DEC,,0.0,16-05-2020,"Jacksonville, Florida, USA",,...,17,0,0,3,00:05:00,3 Rnd (5-5-5),Herb Dean,http://www.ufcstats.com/fight-details/821a1c6f...,1,15.00
4,win,Song Yadong,Marlon Vera,Featherweight,U-DEC,,0.0,16-05-2020,"Jacksonville, Florida, USA",,...,29,4,8,3,00:05:00,3 Rnd (5-5-5),Dan Miragliotta,http://www.ufcstats.com/fight-details/81f2f2d4...,0,15.00
5,win,Miguel Baeza,Matt Brown,Welterweight,KO/TKO,Punch,0.0,16-05-2020,"Jacksonville, Florida, USA",,...,2,7,9,2,00:00:18,3 Rnd (5-5-5),Jason Herzog,http://www.ufcstats.com/fight-details/adac148e...,1,5.30
6,win,Kevin Holland,Anthony Hernandez,Middleweight,KO/TKO,Knees,0.0,16-05-2020,"Jacksonville, Florida, USA",,...,5,5,6,1,00:00:39,3 Rnd (5-5-5),Michael Cardoso,http://www.ufcstats.com/fight-details/51461da4...,1,0.65
7,win,Giga Chikadze,Irwin Rivera,Featherweight,U-DEC,,0.0,16-05-2020,"Jacksonville, Florida, USA",,...,0,0,0,3,00:05:00,3 Rnd (5-5-5),Keith Peterson,http://www.ufcstats.com/fight-details/b1e66cea...,0,15.00
8,win,Nate Landwehr,Darren Elkins,Featherweight,U-DEC,,0.0,16-05-2020,"Jacksonville, Florida, USA",,...,28,0,0,3,00:05:00,3 Rnd (5-5-5),Herb Dean,http://www.ufcstats.com/fight-details/f039c935...,1,15.00
9,win,Cortney Casey,Mara Romero Borella,Women's Flyweight,SUB,Armbar,0.0,16-05-2020,"Jacksonville, Florida, USA",,...,2,1,1,1,00:03:36,3 Rnd (5-5-5),Michael Cardoso,http://www.ufcstats.com/fight-details/906283a2...,0,3.60


# Add in Fighter Information:

In [174]:
fighters = pd.read_csv('Data/fighter.csv', index_col = 0)

In [176]:
player_columns = ['DOB', 'Height', 'Reach', 'Stance', 'Weight',
                 'DOB.1', 'Height.1', 'Reach.1', 'Stance.1', 'Weight.1',]

R_B_combined = pd.DataFrame()

#Given 2 players in a match, retrieve their corresponding personal attributes (height, weight, reach, etc.)

for index, row in combined_df.iterrows():

    R_fighter = row['Fighter']
    B_fighter = row['Fighter.1']

    R_stats = fighters[fighters['Name'] == R_fighter].copy()
    B_stats = fighters[fighters['Name'] == B_fighter].copy()
    
    R_stats.reset_index(drop = True, inplace = True)
    B_stats.reset_index(drop = True, inplace = True)
    
    combined = pd.concat([R_stats, B_stats], axis = 1)
    R_B_combined = pd.concat([R_B_combined, combined], axis = 0)

    
    
R_B_combined.drop('Name', axis = 1, inplace = True)
R_B_combined.columns = player_columns
R_B_combined.reset_index(drop = True, inplace = True)

data = pd.concat([combined_df, R_B_combined], axis = 1)
data.shape
data[['R_fighter', 'Fighter', 'Fighter.1']]

,R_fighter,Fighter,Fighter.1
0,Alistair Overeem,Alistair Overeem,Walt Harris
1,Claudia Gadelha,Claudia Gadelha,Angela Hill
2,Dan Ige,Dan Ige,Edson Barboza
3,Krzysztof Jotko,Eryk Anders,Krzysztof Jotko
4,Song Yadong,Song Yadong,Marlon Vera
5,Miguel Baeza,Matt Brown,Miguel Baeza
6,Kevin Holland,Anthony Hernandez,Kevin Holland
7,Giga Chikadze,Giga Chikadze,Irwin Rivera
8,Nate Landwehr,Darren Elkins,Nate Landwehr
9,Cortney Casey,Cortney Casey,Mara Romero Borella


# Append to existing Fight's Data:

In [177]:
cumulative_fights = pd.read_csv('Data/new_fights.csv', index_col = 0)
#cumulative_fights.drop(np.arange(0,11), inplace = True)
#cumulative_fights.head(15)

,Winner,R_fighter,B_fighter,WEIGHT_CLASS,METHOD,DETAIL,title_bout,date,location,attendance,...,DOB,Height,Reach,Stance,Weight,DOB.1,Height.1,Reach.1,Stance.1,Weight.1
11,win,Glover Teixeira,Anthony Smith,Light Heavyweight,KO/TKO,Punches,0.0,13-05-2020,"Jacksonville, Florida, USA",NaN,...,26-07-1988,76.0,76.0,Orthodox,205.0,28-10-1979,74.0,76.0,Orthodox,205.0
12,win,Ben Rothwell,Ovince Saint Preux,Heavyweight,S-DEC,NaN,0.0,13-05-2020,"Jacksonville, Florida, USA",NaN,...,17-10-1981,76.0,78.0,Orthodox,265.0,08-04-1983,75.0,80.0,Southpaw,205.0
13,win,Drew Dober,Alexander Hernandez,Lightweight,KO/TKO,Punches,0.0,13-05-2020,"Jacksonville, Florida, USA",NaN,...,01-10-1992,69.0,72.0,Orthodox,155.0,19-10-1988,68.0,70.0,Southpaw,155.0
14,win,Ricky Simon,Ray Borg,Bantamweight,S-DEC,NaN,0.0,13-05-2020,"Jacksonville, Florida, USA",NaN,...,31-08-1992,66.0,69.0,Orthodox,135.0,04-08-1993,64.0,63.0,Orthodox,135.0
15,win,Andrei Arlovski,Philipe Lins,Heavyweight,U-DEC,NaN,0.0,13-05-2020,"Jacksonville, Florida, USA",NaN,...,04-02-1979,75.0,77.0,Orthodox,240.0,NaN,NaN,NaN,NaN,NaN
16,win,Thiago Moises,Michael Johnson,Lightweight,SUB,Ankle Lock,0.0,13-05-2020,"Jacksonville, Florida, USA",NaN,...,04-06-1986,61.0,73.0,Southpaw,155.0,23-03-1995,69.0,70.0,Orthodox,155.0
17,win,Sijara Eubanks,Sarah Moras,Women's Bantamweight,U-DEC,NaN,0.0,13-05-2020,"Jacksonville, Florida, USA",NaN,...,27-04-1985,64.0,67.0,NaN,135.0,30-04-1988,67.0,67.0,NaN,135.0
18,win,Omar Morales,Gabriel Benitez,Lightweight,U-DEC,NaN,0.0,13-05-2020,"Jacksonville, Florida, USA",NaN,...,15-06-1988,68.0,71.0,Southpaw,145.0,17-10-1985,61.0,73.0,Orthodox,155.0
19,win,Brian Kelleher,Hunter Azure,Featherweight,KO/TKO,Punch,0.0,13-05-2020,"Jacksonville, Florida, USA",NaN,...,02-03-1992,68.0,69.0,Orthodox,135.0,19-08-1986,66.0,66.0,Switch,135.0
20,win,Chase Sherman,Ike Villanueva,Heavyweight,KO/TKO,Elbow,0.0,13-05-2020,"Jacksonville, Florida, USA",NaN,...,16-11-1989,76.0,78.0,Orthodox,250.0,NaN,NaN,NaN,NaN,NaN


In [178]:
append_new_df = pd.concat([data, cumulative_fights], axis = 0)
append_new_df.reset_index(drop=True, inplace = True)
print(append_new_df.shape)
print(append_new_df[['R_fighter', 'Fighter', 'Fighter.1']])
append_new_df.head(5)


(5564, 77)
             R_fighter           Fighter        Fighter.1
0     Alistair Overeem  Alistair Overeem      Walt Harris
1      Claudia Gadelha   Claudia Gadelha      Angela Hill
2              Dan Ige           Dan Ige    Edson Barboza
3      Krzysztof Jotko       Eryk Anders  Krzysztof Jotko
4          Song Yadong       Song Yadong      Marlon Vera
...                ...               ...              ...
5559    Gerard Gordeau    Gerard Gordeau     Kevin Rosier
5560      Ken Shamrock      Ken Shamrock    Patrick Smith
5561      Royce Gracie      Royce Gracie    Art Jimmerson
5562      Kevin Rosier      Kevin Rosier     Zane Frazier
5563    Gerard Gordeau    Gerard Gordeau       Teila Tuli

[5564 rows x 3 columns]


,Winner,R_fighter,B_fighter,WEIGHT_CLASS,METHOD,DETAIL,title_bout,date,location,attendance,...,DOB,Height,Reach,Stance,Weight,DOB.1,Height.1,Reach.1,Stance.1,Weight.1
0,win,Alistair Overeem,Walt Harris,Heavyweight,KO/TKO,Punches,0.0,16-05-2020,"Jacksonville, Florida, USA",,...,17-05-1980,76.0,80.0,Orthodox,265.0,10-06-1983,77.0,77.0,Southpaw,250.0
1,win,Claudia Gadelha,Angela Hill,Women's Strawweight,S-DEC,,0.0,16-05-2020,"Jacksonville, Florida, USA",,...,07-12-1988,64.0,63.0,Orthodox,115.0,12-01-1985,63.0,64.0,Orthodox,115.0
2,win,Dan Ige,Edson Barboza,Featherweight,S-DEC,,0.0,16-05-2020,"Jacksonville, Florida, USA",,...,06-08-1991,67.0,71.0,Orthodox,145.0,21-01-1986,61.0,75.0,Orthodox,155.0
3,win,Krzysztof Jotko,Eryk Anders,Middleweight,U-DEC,,0.0,16-05-2020,"Jacksonville, Florida, USA",,...,21-04-1987,73.0,75.0,Southpaw,205.0,19-08-1989,73.0,77.0,Southpaw,185.0
4,win,Song Yadong,Marlon Vera,Featherweight,U-DEC,,0.0,16-05-2020,"Jacksonville, Florida, USA",,...,02-12-1997,68.0,67.0,Orthodox,135.0,02-12-1992,68.0,70.0,Switch,135.0


In [180]:
append_new_df.to_csv('Data/new_fights.csv')

# Update Relevant Personals:

In [182]:
new_fight_names = [name for name in data['Fighter']] + [name for name in data['Fighter.1']]

In [183]:
pd.set_option('display.max_rows', 100)

df = append_new_df.copy()
for col in df.columns:
    try:
        df[col] = df[col].astype('float')
    except:
        pass

In [184]:
#Relevant columns when breaking out data for specific fighters

fight_columns = ['Winner', 'R_fighter', 'Fighter', 'KD',
       'Sig. str. %', 'Td %', 'Sub. att', 'Pass', 'Rev.', 'Sig. str. Hits',
       'Sig. str. Attempts', 'Total str. Hits', 'Total str. Attempts',
       'Td Hits', 'Td Attempts', 'Head Hits', 'Head Attempts', 'Body Hits',
       'Body Attempts', 'Leg Hits', 'Leg Attempts', 'Distance Hits',
       'Distance Attempts', 'Clinch Hits', 'Clinch Attempts', 'Ground Hits',
       'Ground Attempts', 'DOB', 'Height', 'Reach',
       'Stance', 'Weight', 'date', 'match_time']

fight_columns_1 = ['Winner','R_fighter','Fighter.1', 'KD.1', 'Sig. str. %.1', 'Td %.1',
       'Sub. att.1', 'Pass.1', 'Rev..1', 'Sig. str. Hits.1',
       'Sig. str. Attempts.1', 'Total str. Hits.1', 'Total str. Attempts.1',
       'Td Hits.1', 'Td Attempts.1', 'Head Hits.1', 'Head Attempts.1',
       'Body Hits.1', 'Body Attempts.1', 'Leg Hits.1', 'Leg Attempts.1',
       'Distance Hits.1', 'Distance Attempts.1', 'Clinch Hits.1',
       'Clinch Attempts.1', 'Ground Hits.1', 'Ground Attempts.1', 'DOB.1', 'Height.1', 'Reach.1', 'Stance.1',
       'Weight.1', 'date', 'match_time']

def get_fight_attributes(index):
    
    temp = df.loc[index]
    return temp[['Fighter', 'Fighter.1', 'title_bout',
                'date', 'location', 'attendance', 'rounds', 'time', 'format', 'referee']]
    

def get_fights(fighter):
    
    data = df[(df['Fighter'] == fighter) | (df['Fighter.1'] == fighter)]
    
    return data


#get_personal returns fight statistics only fighter  
def get_personal(fights, fighter):
    
    a = fights[fights['Fighter'] == fighter]
    a1 = fights[fights['Fighter.1'] == fighter]
    
    a = a[fight_columns].copy()
    a1 = a1[fight_columns_1].copy()

    a1.columns = fight_columns  
    
    combined = pd.concat([a, a1], axis = 0)
    combined.sort_index(ascending = False, inplace = True)
    
    return combined

#Calculate age of fighter given DOB:
def calc_age(personal):
    try:
        dob = dt.strptime(personal['DOB'].iloc[0], '%d-%m-%Y')
        match_dates = personal['date'].apply(lambda x: dt.strptime(x, '%d-%m-%Y'))

        ages = match_dates.apply(lambda x: round(((x - dob).days)/365,2))
    except:
        ages = pd.Series(float(0.0), index=personal.index)
        
    ages = ages.rename('age')

    return ages

#Calculate a fighter's # of Wins & Losses:
def calc_WL(personal, shift = True):
        
    wins = personal['R_fighter'].apply(lambda x: x == personal['Fighter'].iloc[0])
    losses = personal['R_fighter'].apply(lambda x: x != personal['Fighter'].iloc[0])
    draws = pd.Series(0, index = wins.index)
    
    for idx in wins.index:
        if personal['Winner'].loc[idx] != 'win':
            draws[idx] = 1
            wins[idx] = False
            losses[idx] = False
    
    #Winning/Losing Streak Algorith:
    switch = []
    for item in np.arange(1, len(wins)):
        if wins.iloc[item] == wins.iloc[item-1]:
            switch.append(False)
        else:
            switch.append(True)
    
    if wins.iloc[0]:
        streak = [1]
    else:
        streak = [-1]

    for item, swap in zip(wins, switch):
        if swap:
            streak.append(1 * -np.sign(streak[-1]))
        else:
            streak.append(streak[-1] + np.sign(streak[-1])*1)

            
    streak = pd.Series(streak, index = wins.index)
    #End streak algorithm
        
    
    wl = pd.DataFrame([np.cumsum(losses), np.cumsum(wins), np.cumsum(draws)]).T
    wl = pd.concat([wl, streak], axis = 1)
    wl.columns = ['loss', 'win', 'draws', 'streak']
    
    if shift:
        wl = wl.shift(1)
    
    wl.iloc[0, :] = 0

    return wl

#Calculate a fighter's cumulative time in the ring (across all matches):
def calc_time(personal, shift = True):

    time = np.cumsum(personal['match_time'])
    initial = time.iloc[0]
    
    if shift:
        time = time.shift(1)
    
    time.iloc[0] = initial
    time = time.rename('cum_match_time')

    return time

#Calculate cumulative fight statistics for each fighter
def calc_stats(personal, shift = True):
    stat_columns = ['KD', 'Sub. att', 'Pass',
       'Rev.', 'Sig. str. Hits', 'Sig. str. Attempts', 'Total str. Hits',
       'Total str. Attempts', 'Td Hits', 'Td Attempts', 'Head Hits',
       'Head Attempts', 'Body Hits', 'Body Attempts', 'Leg Hits',
       'Leg Attempts', 'Distance Hits', 'Distance Attempts', 'Clinch Hits',
       'Clinch Attempts', 'Ground Hits', 'Ground Attempts']
    
    stats = personal[stat_columns]
    
    cumulative_columns = ['cum_{}'.format(item) for item in stat_columns]
    
    stats = np.cumsum(stats)
    initial = stats.iloc[0, :].copy()
    
    if shift:
        stats = stats.shift(1)
    
    stats.iloc[0, :] = initial
    stats.columns = cumulative_columns
    
    return stats


#The calculated personal is the personal_df with all cumulative statistics included

def get_calculated_personal(fighter, shift = True):
    
    if (len(df[df['Fighter'] == fighter]) == 0) & (len(df[df['Fighter.1'] == fighter]) == 0):
        return None
    
    
    personal = get_personal(df, fighter)
    
    ages = calc_age(personal)
    
    wl = calc_WL(personal, shift)
    round_times = calc_time(personal, shift)
    stats = calc_stats(personal, shift)
    
    final = pd.concat([personal, ages, wl, round_times, stats], axis = 1)
    
    return final



In [185]:
new_personals = []
final_df = pd.DataFrame()

for name in new_fight_names:
    new_personals.append(get_calculated_personal(name))



    
for personal in new_personals:
    final_df = final_df.append(personal.iloc[-1, :])
    
final_df = final_df[new_personals[0].columns]
#final_df
#new_personals    


In [186]:
cumulative_columns = ['cum_KD', 'cum_Sub. att', 'cum_Pass', 'cum_Rev.',
       'cum_Sig. str. Hits', 'cum_Sig. str. Attempts', 'cum_Total str. Hits',
       'cum_Total str. Attempts', 'cum_Td Hits', 'cum_Td Attempts',
       'cum_Head Hits', 'cum_Head Attempts', 'cum_Body Hits',
       'cum_Body Attempts', 'cum_Leg Hits', 'cum_Leg Attempts',
       'cum_Distance Hits', 'cum_Distance Attempts', 'cum_Clinch Hits',
       'cum_Clinch Attempts', 'cum_Ground Hits', 'cum_Ground Attempts']

#Use cumulative statistics / 10 minutes of match time (10 min. is global match time average across all fighters)
for item in cumulative_columns:
    a = round(final_df[item] / final_df['cum_match_time'] * 10,2)
    final_df['avg_{}'.format(item)] = a

In [187]:
drop_columns = ['KD', 'Sig. str. %', 'Td %',
       'Sub. att', 'Pass', 'Rev.', 'Sig. str. Hits', 'Sig. str. Attempts',
       'Total str. Hits', 'Total str. Attempts', 'Td Hits', 'Td Attempts',
       'Head Hits', 'Head Attempts', 'Body Hits', 'Body Attempts', 'Leg Hits',
       'Leg Attempts', 'Distance Hits', 'Distance Attempts', 'Clinch Hits',
       'Clinch Attempts', 'Ground Hits', 'Ground Attempts']

final_df.drop(drop_columns, axis = 1, inplace = True)

sig_str = final_df['cum_Sig. str. Hits'] / final_df['cum_Sig. str. Attempts']
hits = final_df['cum_Total str. Hits'] / final_df['cum_Total str. Attempts']
tds = final_df['cum_Td Hits'] / final_df['cum_Td Attempts']
head = final_df['cum_Head Hits'] / final_df['cum_Head Attempts']
leg = final_df['cum_Leg Hits'] / final_df['cum_Leg Attempts']
body = final_df['cum_Body Hits'] / final_df['cum_Body Attempts']
distance = final_df['cum_Distance Hits'] / final_df['cum_Distance Attempts']
clinch = final_df['cum_Clinch Hits'] / final_df['cum_Clinch Attempts']
ground = final_df['cum_Ground Hits'] / final_df['cum_Ground Attempts']

efficiency_stats = [sig_str, hits, tds, head, leg, body, distance, clinch, ground]
efficiency_columns = ['eff_sig_str', 'eff_hits', 'eff_tds', 'eff_head', 'eff_leg', 'eff_body', 
                      'eff_distance', 'eff_clinch', 'eff_ground']

for item in efficiency_stats:
    item.fillna(0, inplace = True)
    
for name, item in zip(efficiency_columns, efficiency_stats):
    final_df[name] = item
    final_df[name] = final_df[name].apply(lambda x: round(x, 2))

    
drop_columns = ['cum_KD', 'cum_Sub. att', 'cum_Pass', 'cum_Rev.',
       'cum_Sig. str. Hits', 'cum_Sig. str. Attempts', 'cum_Total str. Hits',
       'cum_Total str. Attempts', 'cum_Td Hits', 'cum_Td Attempts',
       'cum_Head Hits', 'cum_Head Attempts', 'cum_Body Hits',
       'cum_Body Attempts', 'cum_Leg Hits', 'cum_Leg Attempts',
       'cum_Distance Hits', 'cum_Distance Attempts', 'cum_Clinch Hits',
       'cum_Clinch Attempts', 'cum_Ground Hits', 'cum_Ground Attempts']

final_df.drop(drop_columns, axis = 1, inplace = True)

In [188]:
final_columns = ['Winner', 'R_fighter', 'Fighter', 'DOB', 'Height', 'Reach', 'Stance',
       'Weight', 'date', 'match_time', 'age', 'loss', 'win', 'draws', 'streak',
       'cum_match_time', 'avg_cum_KD', 'avg_cum_Sub. att', 'avg_cum_Pass',
       'avg_cum_Rev.', 'avg_cum_Sig. str. Hits', 'avg_cum_Sig. str. Attempts',
       'avg_cum_Total str. Hits', 'avg_cum_Total str. Attempts',
       'avg_cum_Td Hits', 'avg_cum_Td Attempts', 'avg_cum_Head Hits',
       'avg_cum_Head Attempts', 'avg_cum_Body Hits', 'avg_cum_Body Attempts',
       'avg_cum_Leg Hits', 'avg_cum_Leg Attempts', 'avg_cum_Distance Hits',
       'avg_cum_Distance Attempts', 'avg_cum_Clinch Hits',
       'avg_cum_Clinch Attempts', 'avg_cum_Ground Hits',
       'avg_cum_Ground Attempts', 'eff_sig_str', 'eff_hits', 'eff_tds',
       'eff_head', 'eff_leg', 'eff_body', 'eff_distance', 'eff_clinch',
       'eff_ground']

final_columns_1 = ['{}.1'.format(item) for item in final_columns]

pd.set_option('display.max_rows', 100)

reformat_final_df = pd.DataFrame()

#For every fight_index, there are exactly two fighers that participated. Combine them into one fight observation:
for item in set(final_df.index):
    temp = pd.DataFrame(final_df.loc[item].iloc[0, :]).T
    temp1 = pd.DataFrame(final_df.loc[item].iloc[1, :]).T

    temp.columns = final_columns
    temp1.columns = final_columns_1
    
    combined = pd.concat([temp, temp1], axis = 1)
    reformat_final_df = pd.concat([reformat_final_df, combined], axis = 0)
    
drop_columns = ['Winner.1', 'R_fighter.1', 'match_time', 'match_time.1', 'DOB', 'DOB.1', 'date', 'date.1']
reformat_final_df.drop(drop_columns, axis = 1, inplace = True)

In [189]:
combined_df['num_rounds'] = combined_df['format'].apply(lambda x: int(x[0]) if x[0] != 'N' else 1)
combined_df.reset_index(drop = True, inplace = True)

keep_columns = ['WEIGHT_CLASS', 'title_bout', 'location', 'attendance', 'num_rounds']
event_attr = combined_df[keep_columns].copy()

reformat_final_df = pd.concat([reformat_final_df, event_attr], axis = 1)

winner = []
for index, row in reformat_final_df.iterrows():
    
    if row['Winner'] != 'win':
        winner.append(99999)
    
    elif (row['R_fighter'] == row['Fighter']):
        winner.append(0)
    else:
        winner.append(1)
        
reformat_final_df['label'] = winner

reformat_final_df['matches'] = reformat_final_df['win'] + reformat_final_df['loss'] + reformat_final_df['draws']+1
reformat_final_df['matches.1'] = reformat_final_df['win.1'] + reformat_final_df['loss.1'] + reformat_final_df['draws.1']+1

In [190]:
reformat_final_df.columns

Index(['Winner', 'R_fighter', 'Fighter', 'Height', 'Reach', 'Stance', 'Weight',
       'age', 'loss', 'win', 'draws', 'streak', 'cum_match_time', 'avg_cum_KD',
       'avg_cum_Sub. att', 'avg_cum_Pass', 'avg_cum_Rev.',
       'avg_cum_Sig. str. Hits', 'avg_cum_Sig. str. Attempts',
       'avg_cum_Total str. Hits', 'avg_cum_Total str. Attempts',
       'avg_cum_Td Hits', 'avg_cum_Td Attempts', 'avg_cum_Head Hits',
       'avg_cum_Head Attempts', 'avg_cum_Body Hits', 'avg_cum_Body Attempts',
       'avg_cum_Leg Hits', 'avg_cum_Leg Attempts', 'avg_cum_Distance Hits',
       'avg_cum_Distance Attempts', 'avg_cum_Clinch Hits',
       'avg_cum_Clinch Attempts', 'avg_cum_Ground Hits',
       'avg_cum_Ground Attempts', 'eff_sig_str', 'eff_hits', 'eff_tds',
       'eff_head', 'eff_leg', 'eff_body', 'eff_distance', 'eff_clinch',
       'eff_ground', 'Fighter.1', 'Height.1', 'Reach.1', 'Stance.1',
       'Weight.1', 'age.1', 'loss.1', 'win.1', 'draws.1', 'streak.1',
       'cum_match_time.1', 'av

In [191]:
#to_viz = pd.read_csv('Data/to_vis.csv', index_col = 0)
#to_viz = to_viz.append(reformat_final_df)
#to_viz.reset_index(drop=True, inplace = True)
#to_viz.to_csv('Data/to_vis.csv')

In [193]:
#Difference is measured as fighter 0 - fighter 1

reformat_final_df['Reach_diff'] = reformat_final_df['Reach'] - reformat_final_df['Reach.1']
reformat_final_df['Weight_diff'] = reformat_final_df['Weight'] - reformat_final_df['Weight.1']
reformat_final_df['Height_diff'] = reformat_final_df['Height'] - reformat_final_df['Height.1']

reformat_final_df['round_age'] = reformat_final_df['age'].apply(lambda x: round(x,0))
reformat_final_df['round_age.1'] = reformat_final_df['age.1'].apply(lambda x: round(x,0))

In [194]:
update_df = pd.read_csv('Data/to_model.csv', index_col = 0)
print(update_df.shape)

(5471, 99)


In [195]:
update_df = pd.concat([reformat_final_df, update_df], axis = 0)
update_df.reset_index(drop = True, inplace = True)
print(update_df.shape)
update_df

(5482, 99)


,Winner,R_fighter,Fighter,Height,Reach,Stance,Weight,age,loss,win,...,attendance,num_rounds,label,matches,matches.1,Reach_diff,Weight_diff,Height_diff,round_age,round_age.1
0,win,Alistair Overeem,Alistair Overeem,76,80,Orthodox,265,40.02,7,10,...,,5,0,18,14,3,15,-1,40.0,37.0
1,win,Claudia Gadelha,Claudia Gadelha,64,63,Orthodox,115,31.46,4,6,...,,3,0,11,15,-1,0,1,31.0,35.0
2,win,Dan Ige,Dan Ige,67,71,Orthodox,145,28.8,1,5,...,,3,0,7,23,-4,-10,6,29.0,34.0
3,win,Krzysztof Jotko,Eryk Anders,73,75,Southpaw,205,33.09,4,5,...,,3,1,10,13,-2,20,0,33.0,31.0
4,win,Song Yadong,Song Yadong,68,67,Orthodox,135,22.47,0,4,...,,3,0,6,14,-3,0,0,22.0,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5477,win,Gerard Gordeau,Gerard Gordeau,77,77.3347,Orthodox,216,34.65,0,1,...,2800,1,0,2,2,-4.39899,-59,1,35.0,0.0
5478,win,Ken Shamrock,Patrick Smith,74,77.5172,Orthodox,225,30.23,0,0,...,2800,1,1,1,1,5.51721,20,1,30.0,30.0
5479,win,Royce Gracie,Royce Gracie,73,73.4723,Southpaw,175,26.94,0,0,...,2800,1,0,1,1,-1.62681,-21,0,27.0,30.0
5480,win,Kevin Rosier,Kevin Rosier,76,81.7337,Orthodox,275,0,0,0,...,2800,1,0,1,1,1.7651,25,-1,0.0,0.0


In [196]:
update_df.to_csv('Data/to_model.csv')